In [3]:
import cv2
import face_recognition
import numpy as np
import requests
import json

def detect_faces_with_dnn_and_send_to_server(image_path, server_url):
    # Load pre-trained model from OpenCV DNN (Caffe model for face detection)
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    # Load the image
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    # Preprocess the image: resize, normalize
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_locations = []

    # Loop over the detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections (confidence threshold can be adjusted)
        if confidence > 0.5:
            # Get the bounding box for the detected face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Append to face locations list
            face_locations.append((startY, endX, endY, startX))

            # Print the bounding box and confidence
            print(f"Face {i+1}: Bounding box [(startX: {startX}, startY: {startY}), (endX: {endX}, endY: {endY})], Confidence: {confidence}")

            # Draw the bounding box around the face
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Check if any faces were detected
    if len(face_locations) > 0:
        # Get the face encodings for the detected faces using face_recognition
        face_encodings = face_recognition.face_encodings(image, face_locations)

        # Prepare the data to send to the server
        data_to_send = {
            'encodings': [encoding.tolist() for encoding in face_encodings]  # Convert numpy arrays to lists
        }

        # Send the face encodings to the server for recognition
        response = requests.post(server_url, json=data_to_send)

        # Print the server's response
        print("Server response:", response.json())

        # Exit the function after the server response to stop further execution
        return
    else:
        print("No faces detected.")

    # If the function has not exited by this point, show the image with detected faces
    cv2.imshow("Detected Faces", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function
image_path = 'jack2.jpeg'
server_url = 'http://127.0.0.1:5000/recognize'
detect_faces_with_dnn_and_send_to_server(image_path, server_url)


Face 1: Bounding box [(startX: 859, startY: 963), (endX: 1547, endY: 2075)], Confidence: 0.9997692704200745
Server response: {'name': 'jack'}
